# 第13章 RAG

## 13.3 RAG 向けに LLM を指示チューニングする

### 13.3.2 指示チューニングしたモデルを LangChain で使う

#### 環境の準備

In [ ]:
!pip install transformers[torch,sentencepiece] langchain langchain-community langchain-huggingface faiss-cpu jq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 381.4 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [ ]:
from huggingface_hub import notebook_login

# Hugging Face Hubにログイン
notebook_login()

In [ ]:
from transformers.trainer_utils import set_seed

# 乱数のシードを設定
set_seed(42)

#### Chat Model の作成

In [ ]:
import torch
from langchain_huggingface import (
    ChatHuggingFace,
    HuggingFacePipeline,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)

# Hugging Face Hubにおけるモデル名を指定
model_name = "llm-book/Swallow-7b-hf-oasst1-21k-ja-aio-retriever"

# モデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# トークナイザを読み込む
tokenizer = AutoTokenizer.from_pretrained(model_name)

# テキスト生成用のパラメータを指定
generation_config = {
    "max_new_tokens": 32,
    "do_sample": False,
    "temperature": None,
    "top_p": None,
}

# テキスト生成を行うパイプラインを作成
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    **generation_config,
)

# パイプラインからLangChainのLLMコンポーネントを作成
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# LLMコンポーネントを元にChat Modelコンポーネントを作成
chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer)

config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

#### Embedding Model の作成

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Hugging Face Hubにおけるモデル名を指定
embedding_model_name = "BAAI/bge-m3"

# モデル名からEmbedding Modelを初期化
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"model_kwargs": {"torch_dtype": torch.float16}},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

#### データストアの構築

In [ ]:
# 検索対象の文書集合のファイルをダウンロード
!wget  \
https://github.com/ghmagazine/llm-book/raw/main/chapter13/docs.json

In [ ]:
from langchain_community.document_loaders import JSONLoader

# JSONファイルから文書を読み込むためのDocument Loaderを初期化
document_loader = JSONLoader(
    file_path="./docs.json",  # 読み込みを行うファイル
    jq_schema=".text",  # 読み込み対象のフィールド
    json_lines=True,  # JSON Lines形式のファイルであることを指定
)

# 文書の読み込みを実行
documents = document_loader.load()

# 読み込まれた文書数を確認
print(len(documents))

103


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 文書を指定した文字数で分割するText Splitterを初期化
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # 分割する最大文字数
    chunk_overlap=100,  # 分割された文書間で重複させる最大文字数
    add_start_index=True,  # 元の文書における開始位置の情報を付与
)

# 文書の分割を実行
split_documents = text_splitter.split_documents(documents)

# 分割後の文書数を確認
print(len(split_documents))

1475


#### 検索対象の文書のベクトルインデックスの作成

In [ ]:
from langchain_community.vectorstores import FAISS

# 分割後の文書と文埋め込みモデルを用いて、Faissのベクトルインデックスを作成
vectorstore = FAISS.from_documents(split_documents, embedding_model)

# ベクトルインデックスに登録された文書数を確認
print(vectorstore.index.ntotal)

1475


#### Retriever コンポーネントの作成

In [ ]:
# ベクトルインデックスを元に文書の検索を行うRetrieverを初期化
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
from pprint import pprint

# 文書の検索を実行
retrieved_documents = retriever.invoke("四国地方で一番高い山は？")

# 検索された文書を確認
pprint(retrieved_documents)

[Document(metadata={'source': '/content/docs.json', 'seq_num': 26, 'start_index': 0}, page_content='この項目に含まれる文字「鎚」は、オペレーティングシステムやブラウザなどの環境により表示が異なります。 石鎚山（いしづちさん、いしづちやま）は、四国山地西部に位置する標高1,982 mの山で、近畿以西を「西日本」とした場合の西日本最高峰で、山頂から望む展望が四国八十八景64番に選定。愛媛県西条市と久万高原町の境界に位置する。 石鉄山、石鈇山、石土山、石槌山とも表記され、伊予の高嶺とも呼ばれる。『日本霊異記』には「石槌山」と記され、延喜式の神名帳（延喜式神名帳）では「石鉄神社」と記されている。前神寺および横峰寺では「石鈇山（しゃくまざん）」とも呼ぶ。'),
 Document(metadata={'source': '/content/docs.json', 'seq_num': 1, 'start_index': 0}, page_content='富士山（ふじさん）は、静岡県（富士宮市、富士市、裾野市、御殿場市、駿東郡小山町）と山梨県（富士吉田市、南都留郡鳴沢村）に跨る活火山である。標高3776.12 m、日本最高峰（剣ヶ峰）の独立峰で、その優美な風貌は日本国外でも日本の象徴として広く知られている。 数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。'),
 Document(metadata={'source': '/content/docs.json', 'seq_num': 96, 'start_index': 0}, page_content='四阿山（あずまやさん）は、長野県と群馬県の県境に跨る山。標高2,354 m。日本百名山の一つに数えられている。吾妻山・吾嬬山（あがつまやま）などとも呼ばれ、嬬恋村では吾妻山が用いられている。 上信国境の山では、浅間山 (2,568m) に次ぐ標高であり志賀高原最高峰、裏岩菅山 (2,341m) より13m高いが、東北最高峰である燧ヶ岳 (2,356m) より2m低い。 約80万年前から30

#### RAG の Chain の構築と実行

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 任意のqueryからメッセージを構築するPrompt Templateを作成
rag_prompt_text = (
    "あなたには今からクイズに答えてもらいます。"
    "問題を与えますので、その解答のみを簡潔に出力してください。\n"
    "また解答の参考になりうるテキストを与えます。"
    "解答を含まない場合もあるのでその場合は無視してください。\n\n"
    "---\n{context}\n---\n\n問題: {query}"
)
rag_prompt_template = ChatPromptTemplate.from_messages(
    [("user", rag_prompt_text)]
)

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

def format_documents_func(documents: list[Document]) -> str:
    """文書のリストを改行で連結した一つの文字列として返す"""
    return "\n\n".join(
        document.page_content for document in documents
    )

# 定義した関数の処理を行うRunnableを作成
format_documents = RunnableLambda(format_documents_func)

In [ ]:
from langchain_core.prompt_values import ChatPromptValue

def chat_model_resp_only_func(
    chat_prompt_value: ChatPromptValue,
) -> str:
    """chat_modelにchat_prompt_valueを入力し、
    出力からモデルの応答部分のみを文字列で返す"""
    chat_prompt = chat_model._to_chat_prompt(
        chat_prompt_value.messages
    )
    chat_output_message = chat_model.invoke(chat_prompt_value)
    response_text = chat_output_message.content[len(chat_prompt) :]
    return response_text

# 定義した関数の処理を行うRunnableを作成
chat_model_resp_only = RunnableLambda(chat_model_resp_only_func)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# RAGの一連の処理を行うChainを作成
rag_chain = (
    {
        "context": retriever | format_documents,
        "query": RunnablePassthrough(),
    }
    | rag_prompt_template
    | chat_model_resp_only
)

In [ ]:
# Chainを実行し、結果を確認
rag_chain_output = rag_chain.invoke("四国地方で一番高い山は？")
print(rag_chain_output)

石鎚山
